In [1]:
# Initial imports.
import sqlalchemy
import numpy as np
from pathlib import Path
from collections import Counter
import pandas as pd
from path import Path
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sqlalchemy import create_engine

In [2]:
# Loading data
file_path = Path("../Data/final_table_ml_stage2.csv")
risk_df = pd.read_csv(file_path)
risk_df.head()

,business_postal_code_1,inspection_id,inspection_date,inspection_score,violation_description,risk_category,neighborhoods,Current Police Districts,population,avg_income
0,94133,835_20180917,September,88,Improper food storage,Low Risk,107,6,"26,827","$40,990.00"
1,94108,905_20190415,April,87,High risk vermin infestation,High Risk,19,6,"13,716","$31,542.00"
2,94118,1203_20170803,August,77,Moderate risk food holding temperature,Moderate Risk,5,8,"38,939","$61,609.00"
3,94109,1345_20170928,September,81,Improper cooling methods,High Risk,105,4,"56,322","$43,444.00"
4,94114,1352_20180620,June,74,Non service animal,Low Risk,38,3,"30,574","$75,727.00"


In [3]:
# Drop inspection_id
risk_df1 = risk_df.drop(columns='violation_description')

In [4]:
# Create our features
X = risk_df1.drop(columns='risk_category')

# Create our target
y = risk_df1['risk_category']
y.value_counts()

Low Risk         9820
Moderate Risk    7599
High Risk        2825
Name: risk_category, dtype: int64

In [5]:
X.describe()

,business_postal_code_1,inspection_score,neighborhoods,Current Police Districts
count,20244.000000,20244.000000,20244.000000,20244.000000
mean,94114.061203,85.344892,57.263485,5.399081
std,9.350442,8.110656,35.014539,2.681820
min,94102.000000,46.000000,1.000000,1.000000
25%,94108.000000,81.000000,28.000000,3.000000
50%,94111.000000,87.000000,53.000000,6.000000
75%,94121.000000,92.000000,96.000000,8.000000
max,94134.000000,100.000000,117.000000,10.000000


In [6]:
months_num = {
   "January": 1,
   "February": 2,
   "March": 3,
   "April": 4,
   "May": 5,
   "June": 6,
   "July": 7,
   "August": 8,
   "September": 9,
   "October": 10,
   "November": 11,
   "December": 12,
}

In [7]:
X["months_num"] = X["inspection_date"].apply(lambda x: months_num[x])

In [8]:
months_num["June"]

6

In [9]:
X

,business_postal_code_1,inspection_id,inspection_date,inspection_score,neighborhoods,Current Police Districts,population,avg_income,months_num
0,94133,835_20180917,September,88,107,6,"26,827","$40,990.00",9
1,94108,905_20190415,April,87,19,6,"13,716","$31,542.00",4
2,94118,1203_20170803,August,77,5,8,"38,939","$61,609.00",8
3,94109,1345_20170928,September,81,105,4,"56,322","$43,444.00",9
4,94114,1352_20180620,June,74,38,3,"30,574","$75,727.00",6
...,...,...,...,...,...,...,...,...,...
20239,94118,68998_20181026,October,94,5,8,"38,939","$61,609.00",10
20240,94112,66584_20180430,April,90,90,9,"73,104","$57,629.00",4
20241,94103,67182_20170731,July,96,53,3,"23,016","$31,131.00",7
20242,94103,70090_20170105,January,93,32,1,"23,016","$31,131.00",1


In [10]:
X = X.drop(columns='inspection_date')
X.head()

,business_postal_code_1,inspection_id,inspection_score,neighborhoods,Current Police Districts,population,avg_income,months_num
0,94133,835_20180917,88,107,6,"26,827","$40,990.00",9
1,94108,905_20190415,87,19,6,"13,716","$31,542.00",4
2,94118,1203_20170803,77,5,8,"38,939","$61,609.00",8
3,94109,1345_20170928,81,105,4,"56,322","$43,444.00",9
4,94114,1352_20180620,74,38,3,"30,574","$75,727.00",6


In [11]:
X_encoded = pd.get_dummies(X)
X_encoded

,business_postal_code_1,inspection_score,neighborhoods,Current Police Districts,months_num,inspection_id_1000_20171002,inspection_id_1000_20180523,inspection_id_1000_20190617,inspection_id_1002_20170928,inspection_id_1002_20180509,...,"avg_income_$61,609.00","avg_income_$61,776.00","avg_income_$63,983.00","avg_income_$66,627.00","avg_income_$75,727.00","avg_income_$76,044.00","avg_income_$80,959.00","avg_income_$84,710.00","avg_income_$88,976.00","avg_income_$95,313.00"
0,94133,88,107,6,9,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,94108,87,19,6,4,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,94118,77,5,8,8,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
3,94109,81,105,4,9,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,94114,74,38,3,6,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20239,94118,94,5,8,10,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
20240,94112,90,90,9,4,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
20241,94103,96,53,3,7,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
20242,94103,93,32,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [12]:
data_scaler = StandardScaler()

In [13]:
X_encoded_scaled = data_scaler.fit_transform(X_encoded)
X_encoded_scaled[:1]

array([[ 2.02549393,  0.3273685 ,  1.42048832, ..., -0.1686698 ,
        -0.18869199, -0.10262856]])

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X_encoded_scaled, y, random_state = 1)

In [15]:
y_train.value_counts()

Low Risk         7323
Moderate Risk    5724
High Risk        2136
Name: risk_category, dtype: int64

In [16]:
# Creating a StandardScaler instance.
scaler = StandardScaler()
# Fitting the Standard Scaler with the training data.
X_scaler = scaler.fit(X_train)

# Scaling the data.
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [17]:
# Create a random forest classifier.
rf_model = RandomForestClassifier(n_estimators=128, random_state=78) 

In [18]:
# Fitting the model
rf_model = rf_model.fit(X_train_scaled, y_train)

In [19]:
# Making predictions using the testing data.
predictions = rf_model.predict(X_test_scaled)

In [20]:
# Calculating the confusion matrix.
cm = confusion_matrix(y_test, predictions)

# Create a DataFrame from the confusion matrix.
cm_df = pd.DataFrame(
    cm, index=["Actual 0", "Actual 1", "Actual 2"], columns=["Predicted 0", "Predicted 1", "Actual 2"])

cm_df

,Predicted 0,Predicted 1,Actual 2
Actual 0,88,354,247
Actual 1,187,1468,842
Actual 2,205,1071,599


In [21]:
# Calculating the accuracy score.
acc_score = accuracy_score(y_test, predictions)

In [22]:
# Displaying results
print("Confusion Matrix")
display(cm_df)
print(f"Accuracy Score : {acc_score}")
print("Classification Report")
print(classification_report(y_test, predictions))

Confusion Matrix


,Predicted 0,Predicted 1,Actual 2
Actual 0,88,354,247
Actual 1,187,1468,842
Actual 2,205,1071,599


Accuracy Score : 0.4258051768425212
Classification Report
               precision    recall  f1-score   support

    High Risk       0.18      0.13      0.15       689
     Low Risk       0.51      0.59      0.54      2497
Moderate Risk       0.35      0.32      0.34      1875

     accuracy                           0.43      5061
    macro avg       0.35      0.35      0.34      5061
 weighted avg       0.41      0.43      0.41      5061



In [23]:
# Calculate feature importance in the Random Forest model.
importances = rf_model.feature_importances_
importances

array([0.00581791, 0.11732431, 0.03374548, ..., 0.00039365, 0.00076937,
       0.00034638])

In [24]:
# We can sort the features by their importance.
sorted(zip(rf_model.feature_importances_, X_encoded.columns), reverse=True)

[(0.11732431078290784, 'inspection_score'),
 (0.05661621692652888, 'months_num'),
 (0.03374548437686346, 'neighborhoods'),
 (0.011981358198696567, 'Current Police Districts'),
 (0.005817907021290756, 'business_postal_code_1'),
 (0.0011132489923717762, 'avg_income_$40,990.00 '),
 (0.0009557513303001428, 'population_26,827'),
 (0.0008785591119271738, 'avg_income_$31,131.00 '),
 (0.0008290616283369898, 'population_23,016'),
 (0.0008010826335976236, 'population_30,574'),
 (0.0007693694277620893, 'avg_income_$88,976.00 '),
 (0.0007565586794117849, 'population_56,322'),
 (0.0007516078590725314, 'avg_income_$75,727.00 '),
 (0.0007171257695202897, 'population_74,633'),
 (0.0007113599608485185, 'avg_income_$53,795.00 '),
 (0.0006944238417332701, 'avg_income_$43,444.00 '),
 (0.0006859647934069238, 'avg_income_$22,351.00 '),
 (0.000681137594978069, 'avg_income_$61,362.00 '),
 (0.000677894596016852, 'population_2,058'),
 (0.0006554059999972617, 'avg_income_$63,983.00 '),
 (0.0006528174812814638, '